In [1]:
#%matplotlib inline
import torch
from IPython import display
from matplotlib import pyplot as plt
import numpy as np
import random

### 3.2.1 生成数据集
训练集样本为 1000，输入个数（特征数）为 2，线性回归模型真实权重 $w = [2, -3.4]$ 和偏差 $b=4.2$，以及一个随机的噪声项 $\epsilon$ $$$$
$$y = Xw+b+\epsilon$$

In [2]:
print("下面生成数据集：\n")

num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.randn(num_examples, num_inputs,
                       dtype=torch.float32)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
# 加入噪声项，服从均值为0、标准差为0.01的正态分布
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()),
                       dtype=torch.float32)
print(features[0], labels[0])

下面生成数据集：

tensor([ 0.2460, -0.6750]) tensor(6.9843)


In [3]:
print("通过生成第二个特征和标签的散点图，观察两者间的线性关系")
def use_svg_display():
    # 用矢量图显示
    display.set_matplotlib_formats('svg')
    
def set_figsize(figsize=(3.5, 2.5)):
    use_svg_display()
    # 设置图的尺寸
    plt.rcParams['figure.figsize'] = figsize
    
#set_figsize()
#plt.scatter(features[:, 1].numpy(), labels.numpy(), 1)

通过生成第二个特征和标签的散点图，观察两者间的线性关系


### 3.2.2 读取数据
定义一个函数，每次返回一个 batch_size 的随机样本的特征和标签

In [4]:
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        j = torch.LongTensor(indices[i: min(i+batch_size, num_examples)])
        yield features.index_select(0, j), labels.index_select(0, j)

In [5]:
print("读取第一个小批量的数据样本并打印，每个样本形状为（10， 2）\n")
batch_size = 10

for X, y in data_iter(batch_size, features, labels):
    print(X, y)
    break

读取第一个小批量的数据样本并打印，每个样本形状为（10， 2）

tensor([[-0.3836,  0.6488],
        [-0.1642,  0.9719],
        [ 0.7399, -0.3281],
        [-0.2524,  0.4114],
        [ 0.1905,  1.3425],
        [-0.6931, -0.3858],
        [ 0.4914,  1.1642],
        [-1.0843,  0.3776],
        [-0.9980, -2.0048],
        [-1.3376, -0.2638]]) tensor([1.2217, 0.5771, 6.7937, 2.3090, 0.0119, 4.1270, 1.2223, 0.7297, 9.0266,
        2.4165])


### 3.2.3 初始化参数模型

In [6]:
print("都初始化为均值为 0， 标准差为 0.01的正态随机数，偏差初始为 0\n")

w = torch.tensor(np.random.normal(0, 0.01, (num_inputs, 1)), dtype=torch.float32)
b = torch.zeros(1, dtype=torch.float32)

# 之后模型训练中，需要对这些参数求梯度来迭代参数的值，需要设置 requires_grad=True
w.requires_grad_(requires_grad=True)
b.requires_grad_(requires_grad=True)

都初始化为均值为 0， 标准差为 0.01的正态随机数，偏差初始为 0



tensor([0.], requires_grad=True)

### 3.2.4 定义模型

In [7]:
def linreg(X, w, b):
    # 使用 mm 函数做矩阵乘法
    return torch.mm(X, w) + b

### 3.2.5 定义损失函数

In [8]:
print("使用平方损失来定义损失函数\n")
def squared_loss(y_hat, y):
    return (y_hat - y.view(y_hat.size())) ** 2 / 2

使用平方损失来定义损失函数



### 3.2.6 定义优化算法

In [10]:
def sgd(params, lr, batch_size):
    for param in params:
        # 这里更改param时用的是param.data
        param.data -= lr * param.grad / batch_size 

### 3.2.7 训练模型

In [12]:
lr = 0.03
num_epochs = 3
net = linreg
loss = squared_loss


for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        l = loss(net(X, w, b), y).sum()
        l.backward()
        sgd([w, b], lr, batch_size)
        
        # don't forget 梯度清零
        w.grad.data.zero_()
        b.grad.data.zero_()
    train_l = loss(net(features, w, b), labels)
    print('epoch %d, loss %f' %(epoch+1, train_l.mean().item()))

epoch 1, loss 0.033419
epoch 2, loss 0.000124
epoch 3, loss 0.000050


In [14]:
print("对比一下学到的参数和真实的参数\n")
print(true_w, '\n', w)
print(true_b, '\n', b)

对比一下学到的参数和真实的参数

[2, -3.4] 
 tensor([[ 1.9995],
        [-3.3999]], requires_grad=True)
4.2 
 tensor([4.2006], requires_grad=True)
